In [1]:
import base64
import json
import random

import pandas as pd
import requests
import tensorflow as tf

In [2]:
df = pd.read_csv("data/AIDS_Classification.csv")
y = df.pop("infected")

columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs


{'time': 522.0,
 'trt': 0.0,
 'age': 30.0,
 'wtkg': 63.47953,
 'hemo': 0.0,
 'homo': 0.0,
 'drugs': 0.0,
 'karnof': 100.0,
 'oprior': 0.0,
 'z30': 1.0,
 'preanti': 867.0,
 'race': 0.0,
 'gender': 0.0,
 'str2': 1.0,
 'strat': 1.0,
 'symptom': 0.0,
 'treat': 0.0,
 'offtrt': 1.0,
 'cd40': 256.0,
 'cd420': 393.0,
 'cd80': 749.0,
 'cd820': 1092.0}

In [3]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )


def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )


def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )


In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if keys == "wtkg":
            feature_spec[keys] = float_feature(float(values))
        else:
            feature_spec[keys] = int_feature(int(values))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [5]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = (
        "https://aids-prediction-production.up.railway.app/v1/models/aids-model:predict"
    )
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.6:
        return "Not Infected"
    else:
        return "Infected"

In [6]:
make_predictions(inputs)

'Infected'

In [7]:
label

1